In [2]:
import numpy as np
from scipy.sparse import csr_matrix
# Load all k-mers
with open("all_kmers.txt") as f:
    all_kmers = [line.strip() for line in f]
# Load genome IDs
with open("resistantgenomes.txt") as f:
    resistant_genomes = [line.strip() for line in f]
with open("susgenomes.txt") as f:
    susceptible_genomes = [line.strip() for line in f]
# Load k-mer files for resistant and susceptible genomes
with open("kmers_resistant.txt") as f:
    resistant_kmers = set(line.split()[0] for line in f)
with open("kmers_susceptible.txt") as f:
    susceptible_kmers = set(line.split()[0] for line in f)

n_genomes = len(resistant_genomes) + len(susceptible_genomes)
n_kmers = len(all_kmers)
kmer_index = {kmer: idx for idx, kmer in enumerate(all_kmers)}
binary_matrix_sparse = csr_matrix((n_genomes, n_kmers), dtype=int)


In [3]:
#Matrix for resistant genomes (1 for present k-mers)
for i, genome_id in enumerate(resistant_genomes):
    relevant_kmers = list(resistant_kmers.intersection(all_kmers))  # K-mers present in this genome
    indices = [kmer_index[kmer] for kmer in relevant_kmers]
    binary_matrix_sparse[i, indices] = 1

#matrix for susceptible genomes (1 for present k-mers, 0 for absent)
for i, genome_id in enumerate(susceptible_genomes, start=len(resistant_genomes)):
    relevant_kmers = list(susceptible_kmers.intersection(all_kmers))
    indices = [kmer_index[kmer] for kmer in relevant_kmers]
    binary_matrix_sparse[i, indices] = 1

In [4]:
# Invert the susceptible matrix rows
susceptible_rows = slice(len(resistant_genomes), n_genomes)

susceptible_matrix_dense = binary_matrix_sparse[susceptible_rows, :].todense()  # Convert to dense
susceptible_matrix_dense = 1 - susceptible_matrix_dense  # Invert the values
binary_matrix_sparse[susceptible_rows, :] = csr_matrix(susceptible_matrix_dense)  # Convert back to sparse


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/sparse/_index.py:142: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)


In [5]:
genomes = resistant_genomes + susceptible_genomes
n_genomes = binary_matrix_sparse.shape[0]  # Number of genomes
n_kmers = binary_matrix_sparse.shape[1]  # Number of k-mers
#equal weight for all genomes initially)
prob_array = np.ones(n_genomes) / n_genomes

# Genome labels (1 for resistant, 0 for susceptible)
labels = np.array([1 if genome in resistant_genomes else 0 for genome in genomes])
n_rounds = 10  # boosting rounds


In [6]:
import numpy as np

def find_informative_kmers(binary_matrix_sparse, labels, prob_array, n_kmers, n_rounds, batch_size):
    best_kmers = []
    alpha_values = []
    for m in range(n_rounds):
        weighted_errors = []
        # Process k-mers in batches
        for batch_start in range(0, n_kmers, batch_size):
            batch_end = min(batch_start + batch_size, n_kmers)
            batch_kmers = range(batch_start, batch_end)
            # Calculate weighted error for each k-mer in the current batch
            for k in batch_kmers:
                kmer_column = binary_matrix_sparse[:, k]
                # Access the non-zero elements of the sparse column
                non_zero_indices = kmer_column.nonzero()[0]
                # Calculate the weighted error
                incorrect_classifications = (
                    kmer_column[non_zero_indices].toarray().flatten() != labels[non_zero_indices]
                )
                weighted_error = np.sum(prob_array[non_zero_indices][incorrect_classifications])
                weighted_errors.append((k, weighted_error))

        # Find the best k-mer
        best_kmer_idx, best_weighted_error = min(weighted_errors, key=lambda x: x[1])
        best_kmers.append(best_kmer_idx)

        # Compute the weight of the best k-mer
        alpha = 0.5 * np.log((1 - best_weighted_error) / max(best_weighted_error, 1e-10))  # Avoid division by 0
        alpha_values.append(alpha)

        # Update the probability array
        kmer_column = binary_matrix_sparse[:, best_kmer_idx]

        # Update probabilities for non-zero elements of the column
        non_zero_indices = kmer_column.nonzero()[0]

        for i in non_zero_indices:
            if kmer_column[i] == labels[i]:  # If classified correctly
                prob_array[i] *= np.exp(-alpha)  # Decrease probability for correct classifications
            else:  # If classified incorrectly
                prob_array[i] *= np.exp(alpha)  # Increase probability for incorrect classifications

        # Normalize the probability array (to sum to 1)
        prob_array /= np.sum(prob_array)

        print(f"Round {m + 1}: Best k-mer = {best_kmer_idx}, Weighted error = {best_weighted_error:.4f}")

    return best_kmers, alpha_values


In [7]:
## testing the data to see how long the algorith would take according to davis et al 
# i.e 10 rounds of boosting to find the the most informative kmers across 100 genomes
import time
from scipy.sparse import random as sparse_random
# Test case parameters
n_kmers_test = 100000  #
n_rounds_test = 5  # Number of AdaBoost rounds in the test case
batch_size_test = 10000  # Batch size for processing k-mers

# Generate synthetic test data
binary_matrix_sparse_test = sparse_random(n_genomes, n_kmers_test, format='csr', dtype=int)
prob_array_test = np.ones(n_genomes) / n_genomes  # Initialize with equal probabilities

# Call the function with test data
start_time = time.time()
best_kmers, alpha_values = find_informative_kmers(
    binary_matrix_sparse=binary_matrix_sparse_test,
    labels=labels,
    prob_array=prob_array_test,
    n_kmers=n_kmers_test,
    n_rounds=n_rounds_test,
    batch_size=batch_size_test
)
elapsed_time = time.time() - start_time

# Output results for the test case
print(f"\nTest case completed in {elapsed_time:.2f} seconds for {n_kmers_test} k-mers and {n_rounds_test} rounds.")

# Estimate total time for full dataset (12 million k-mers and 10 rounds)
n_kmers_full = n_kmers
n_rounds_full = 10
estimated_time_full = elapsed_time * (n_kmers_full / n_kmers_test) * (n_rounds_full / n_rounds_test)

print(f"Estimated time for full dataset (12 million k-mers, 10 rounds): {estimated_time_full / 3600:.2f} hours.")


Round 1: Best k-mer = 3, Weighted error = 0.0000
Round 2: Best k-mer = 3, Weighted error = 0.0000
Round 3: Best k-mer = 3, Weighted error = 0.0000
Round 4: Best k-mer = 3, Weighted error = 0.0000
Round 5: Best k-mer = 3, Weighted error = 0.0000

Test case completed in 48.64 seconds for 100000 k-mers and 5 rounds.
Estimated time for full dataset (12 million k-mers, 10 rounds): 3.29 hours.


In [8]:
k_subset = 100000  # Number of k-mers to use in the test
binary_matrix_subset = binary_matrix_sparse[:, :k_subset]  # Use all genomes but subset k-mers

find_informative_kmers(binary_matrix_subset, labels, prob_array, k_subset, n_rounds,10000)


/var/folders/m3/sndtdmls1qg5rnrdkjvx8qf40000gn/T/ipykernel_64194/3513499716.py:4: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  find_informative_kmers(binary_matrix_subset, labels, prob_array, k_subset, n_rounds,10000)


Round 1: Best k-mer = 0, Weighted error = 0.4917
Round 2: Best k-mer = 0, Weighted error = 0.4961
Round 3: Best k-mer = 0, Weighted error = 0.4982
Round 4: Best k-mer = 0, Weighted error = 0.4991
Round 5: Best k-mer = 0, Weighted error = 0.4996
Round 6: Best k-mer = 0, Weighted error = 0.4998
Round 7: Best k-mer = 0, Weighted error = 0.4999
Round 8: Best k-mer = 0, Weighted error = 0.5000
Round 9: Best k-mer = 0, Weighted error = 0.5000
Round 10: Best k-mer = 0, Weighted error = 0.5000


([0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [np.float64(0.016668210133795856),
  np.float64(0.007791841807889995),
  np.float64(0.0036453155567761626),
  np.float64(0.0017060411673970306),
  np.float64(0.0007985794871820327),
  np.float64(0.0003738363148424956),
  np.float64(0.00017500926807627093),
  np.float64(8.19309899634491e-05),
  np.float64(3.8356494585317996e-05),
  np.float64(1.7956896563988926e-05)])

In [9]:


def classify(binary_matrix_sparse, genomes, best_kmer, alpha_value):
    
    predictions = []
    
    # Iterate through each genome
    for genome_idx in range(binary_matrix_sparse.shape[0]):
        weighted_vote = 0
        
        # Check if the selected k-mer is present in the genome
        kmer_presence = binary_matrix_sparse[genome_idx, best_kmer]
        
        # Add or subtract alpha value based on presence/absence of k-mer
        if kmer_presence == 1:
            weighted_vote += alpha_value  # Positive vote if k-mer is present
        else:
            weighted_vote -= alpha_value  # Negative vote if k-mer is absent
        
        # Classification decision based on weighted vote
        if weighted_vote > 0:
            predictions.append((genomes[genome_idx], 'Resistant'))
        else:
            predictions.append((genomes[genome_idx], 'Susceptible'))
    
    return predictions
   

In [10]:
bestKmer = best_kmers[0]
alpha_value = alpha_values[0]
kmer_size=31
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
predictions = classify(binary_matrix_sparse, genomes, bestKmer, alpha_value)
predictions_binary = [1 if prediction == 'Resistant' else 0 for _, prediction in predictions]

# Calculate accuracy
accuracy = accuracy_score(labels, predictions_binary)
precision = precision_score(labels, predictions_binary)
recall = recall_score(labels, predictions_binary)
f1 = f1_score(labels, predictions_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.0333
Precision: 0.0635
Recall: 0.0656
F1 Score: 0.0645


In [11]:
batchsize=100000
find_informative_kmers(binary_matrix_sparse,labels,prob_array,n_kmers,n_rounds,batchsize)